This code is mostly from Andrew Johnston, shared across our organization's slack.

In [2]:
import folium.plugins
import requests
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd
from rasterio.crs import CRS
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

# Get the data

The hard part is getting the data. Lots of nuance that was figured out by Andrew Johnston. Uses CMR directly.

In [3]:
import json
import xml.etree.ElementTree as ET

import requests

COLLECTION_CONCEPT_ID = 'C1595422627-ASF'
CMR_URL = 'https://cmr.earthdata.nasa.gov/search/granules.echo10'

def parse_echo10(echo10_xml: str):
    granules = []
    root = ET.fromstring(echo10_xml)
    for granule in root.findall('result/Granule'):
        g = {
            'product_id': granule.find('GranuleUR').text,
            'product_version': granule.find('GranuleUR').text.split('-')[-1],
            'reference_scenes': [],
            'secondary_scenes': []
        }
        for input_granule in granule.findall('InputGranules/InputGranule'):
            input_granule_type, input_granule_name = input_granule.text.split(' ')
            if input_granule_type == '[Reference]':
                g['reference_scenes'].append(input_granule_name)
            else:
                g['secondary_scenes'].append(input_granule_name)
        granules.append(g)
    return granules


def get_ref_secondary_scene(gunw_id: str):
    session = requests.Session()
    search_params = {
        'provider': 'ASF',
        'collection_concept_id': COLLECTION_CONCEPT_ID,
        'page_size': 2000,
        'readable_granule_name': gunw_id
    }
    headers = {}
    products = []

    while True:
        response = session.get(CMR_URL, params=search_params, headers=headers)
        response.raise_for_status()

        parsed_results = parse_echo10(response.text)
        products.extend(parsed_results)

        if 'CMR-Search-After' not in response.headers:
            break
        headers = {'CMR-Search-After': response.headers['CMR-Search-After']}

    return products

In [3]:
gunw_id = 'S1-GUNW-D-R-115-tops-20220504_20220422-141610-00124W_00038N-PP-0442-v2_0_5'
print('California', gunw_id)
get_ref_secondary_scene(gunw_id)

California S1-GUNW-D-R-115-tops-20220504_20220422-141610-00124W_00038N-PP-0442-v2_0_5


[{'product_id': 'S1-GUNW-D-R-115-tops-20220504_20220422-141610-00124W_00038N-PP-0442-v2_0_5',
  'product_version': 'v2_0_5',
  'reference_scenes': ['S1A_IW_SLC__1SDV_20220504T141557_20220504T141624_043062_05246D_3C67'],
  'secondary_scenes': ['S1A_IW_SLC__1SDV_20220422T141557_20220422T141624_042887_051EA4_CD2E']}]

In [4]:
gunw_id = 'S1-GUNW-D-R-044-tops-20210925_20200930-172127-00169W_00053N-PP-812d-v2_0_5'
print('Aleutian', gunw_id)
get_ref_secondary_scene(gunw_id)

Aleutian S1-GUNW-D-R-044-tops-20210925_20200930-172127-00169W_00053N-PP-812d-v2_0_5


[{'product_id': 'S1-GUNW-D-R-044-tops-20210925_20200930-172127-00169W_00053N-PP-812d-v2_0_5',
  'product_version': 'v2_0_5',
  'reference_scenes': ['S1A_IW_SLC__1SDV_20210925T172118_20210925T172137_039841_04B68A_7D82'],
  'secondary_scenes': ['S1A_IW_SLC__1SDV_20200930T172045_20200930T172114_034591_0406F4_8C2F',
   'S1A_IW_SLC__1SDV_20200930T172112_20200930T172131_034591_0406F4_17D7']}]

In [5]:
gunw_id = 'S1-GUNW-D-R-169-tops-20220520_20220508-071114-00020W_00027N-PP-f4ed-v2_0_5'
print('La Palma', gunw_id)
get_ref_secondary_scene(gunw_id)

La Palma S1-GUNW-D-R-169-tops-20220520_20220508-071114-00020W_00027N-PP-f4ed-v2_0_5


[{'product_id': 'S1-GUNW-D-R-169-tops-20220520_20220508-071114-00020W_00027N-PP-f4ed-v2_0_5',
  'product_version': 'v2_0_5',
  'reference_scenes': ['S1A_IW_SLC__1SDV_20220520T071101_20220520T071128_043291_052B7B_7B63'],
  'secondary_scenes': ['S1A_IW_SLC__1SDV_20220508T071100_20220508T071127_043116_05262A_AA24']}]

In [6]:
gunw_id = 'S1-GUNW-A-R-158-tops-20220308_20220224-123254-00080E_00035N-PP-4300-v2_0_5'
print('Issue 60', gunw_id)
get_ref_secondary_scene(gunw_id)

Issue 60 S1-GUNW-A-R-158-tops-20220308_20220224-123254-00080E_00035N-PP-4300-v2_0_5


[{'product_id': 'S1-GUNW-A-R-158-tops-20220308_20220224-123254-00080E_00035N-PP-4300-v2_0_5',
  'product_version': 'v2_0_5',
  'reference_scenes': ['S1A_IW_SLC__1SDV_20220308T123241_20220308T123307_042230_05086E_4B83'],
  'secondary_scenes': ['S1A_IW_SLC__1SDV_20220224T123241_20220224T123308_042055_05027A_A8B5']}]

In [6]:
gunw_id = 'S1-GUNW-A-R-043-tops-20221002_20220908-151531-00041E_00027N-PP-e567-v2_0_5'
print('Arabia with Ionosphere', gunw_id)
get_ref_secondary_scene(gunw_id)

Arabia with Ionosphere S1-GUNW-A-R-043-tops-20221002_20220908-151531-00041E_00027N-PP-e567-v2_0_5


[{'product_id': 'S1-GUNW-A-R-043-tops-20221002_20220908-151531-00041E_00027N-PP-e567-v2_0_5',
  'product_version': 'v2_0_5',
  'reference_scenes': ['S1A_IW_SLC__1SDV_20221002T151520_20221002T151543_045265_056931_E517'],
  'secondary_scenes': ['S1A_IW_SLC__1SDV_20220908T151520_20220908T151542_044915_055D68_78EC']}]

In [8]:
gunw_id = 'S1-GUNW-A-R-041-tops-20170817_20160717-120015-390925N_37050N-PP-ed21-v2_0_4'
print('Tibet Issues 0', gunw_id)
get_ref_secondary_scene(gunw_id)

Tibet Issues 0 S1-GUNW-A-R-041-tops-20170817_20160717-120015-390925N_37050N-PP-ed21-v2_0_4


[{'product_id': 'S1-GUNW-A-R-041-tops-20170817_20160717-120015-390925N_37050N-PP-ed21-v2_0_4',
  'product_version': 'v2_0_4',
  'reference_scenes': ['S1A_IW_SLC__1SDV_20170817T120001_20170817T120028_017963_01E230_A23A-local'],
  'secondary_scenes': ['S1A_IW_SLC__1SSV_20160717T115946_20160717T120014_012188_012E84_F684-local',
   'S1A_IW_SLC__1SSV_20160717T120012_20160717T120039_012188_012E84_4198-local']}]

In [12]:
gunw_id = 'S1-GUNW-A-R-041-tops-20141231_20141207-115927-37270N_35394N-PP-d9a6-v2_0_4'
print('Tibet Issues 1', gunw_id)
get_ref_secondary_scene(gunw_id)

Tibet Issues 1 S1-GUNW-A-R-041-tops-20141231_20141207-115927-37270N_35394N-PP-d9a6-v2_0_4


[{'product_id': 'S1-GUNW-A-R-041-tops-20141231_20141207-115927-37270N_35394N-PP-d9a6-v2_0_4',
  'product_version': 'v2_0_4',
  'reference_scenes': ['S1A_IW_SLC__1SSV_20141231T115913_20141231T115940_003963_004C50_1CEA-local'],
  'secondary_scenes': ['S1A_IW_SLC__1SSV_20141207T115849_20141207T115916_003613_00445F_3EA7-local',
   'S1A_IW_SLC__1SSV_20141207T115914_20141207T115941_003613_00445F_AF5A-local',
   'S1A_IW_SLC__1SSV_20141207T115939_20141207T120007_003613_00445F_BA69-local']}]

In [11]:
gunw_id = 'S1-GUNW-A-R-041-tops-20141231_20141207-115952-38760N_36887N-PP-7560-v2_0_1'
print('Tibet Issues OK', gunw_id)
get_ref_secondary_scene(gunw_id)

Tibet Issues 0 S1-GUNW-A-R-041-tops-20141231_20141207-115952-38760N_36887N-PP-7560-v2_0_1


[{'product_id': 'S1-GUNW-A-R-041-tops-20141231_20141207-115952-38760N_36887N-PP-7560-v2_0_1',
  'product_version': 'v2_0_1',
  'reference_scenes': ['S1A_IW_SLC__1SSV_20141231T115938_20141231T120006_003963_004C50_DB7D'],
  'secondary_scenes': ['S1A_IW_SLC__1SSV_20141207T115914_20141207T115941_003613_00445F_AF5A',
   'S1A_IW_SLC__1SSV_20141207T115939_20141207T120007_003613_00445F_BA69',
   'S1A_IW_SLC__1SSV_20141207T120005_20141207T120031_003613_00445F_F430']}]